##Named Entity Recognition (NER) using a classifier

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

### Import the training and development data

In [ ]:
wnuttrain = 'https://storage.googleapis.com/wnut-2017_ner-shared-task/wnut17train_clean_tagged.txt'
train = pd.read_table(wnuttrain, header=None, names=['token', 'label', 'bio_only', 'upos'])

In [ ]:
wnutdev = 'https://storage.googleapis.com/wnut-2017_ner-shared-task/wnut17dev_clean_tagged.txt'
dev = pd.read_table(wnutdev, header=None, names=['token', 'label', 'bio_only', 'upos'])

###Devise function to exctract all suitable features and store them in copies of the training and development dataframes


In [ ]:
# in order to convert POS tags to integers: get the UPOS tagset
pos_vocab = train.upos.unique().tolist()

# feature 1: convert POS-tags to integers
def pos_index(pos):
  ind = pos_vocab.index(pos)
  return ind

# feature 2: is this a proper noun?
def is_propn(pos):
  resp = False
  if pos=='PROPN':
    resp = True
  return resp

#  feature 3: is the first character a capital letter?
def title_case(tok):
  resp = False
  if tok[0].isupper():
    resp = True  # thanks Archie Barrett for spotting a typo here!
  return resp

# feature 4: token length
def tok_len(tok):
  tok_len = len(tok)
  return tok_len

# features 5, 6, 7 extracted after token i s turned into a spacy token:  
  # spacy tag (extended pos tagging)
  # is the token comprise of alphabetical characters only
  # is the token a stop word
def get_spacy_doc(t):
  ret = ''
  doc = nlp(t)
  for w in doc:
    ret = w
  return ret

# training labels: convert BIO to integers
def bio_index(bio):
  ind = 10
  if bio=='B':
    ind = 0
  elif bio=='I':
    ind = 1
  elif bio=='O':
    ind = 2
  return ind

# pass a data frame through our feature extractor
def extract_features(txt_orig, bio=True):
  txt = txt_orig.copy()
  txt = txt.dropna()

  txt['pos_indices'] = [pos_index(u) for u in txt['upos']]
  txt['is_propn'] = [is_propn(u) for u in txt['upos']]
  txt['title_case'] = [title_case(str(t)) for t in txt['token']]
  txt['token_length'] = [tok_len(str(t)) for t in txt['token']]
  txt['docs'] = [get_spacy_doc(str(t)) for t in txt['token']]
  txt['spacy_tag'] = [t.tag for t in txt['docs']]
  txt['is_alpha'] = [t.is_alpha for t in txt['docs']]
  txt['is_stop'] = [t.is_stop for t in txt['docs']]

  if bio==True:
    bioints = [bio_index(b) for b in txt['bio_only']]
    txt['bio_only'] = bioints

  return txt

###Extract the features for each dataset

In [ ]:
train_copy = extract_features(train)

In [ ]:
dev_copy = extract_features(dev)

###Prepare the data for training: separate into feature vectors (X) and labels (y).
Use oversampling for the training data to account for the significantly larger number of O labels

In [ ]:
smote = SMOTE(sampling_strategy='auto')

In [ ]:
X_train = train_copy.drop(['token', 'label', 'bio_only', 'upos', 'docs'], axis=1)
y_train = train_copy['bio_only']

In [ ]:
X_train, y_train = smote.fit_sample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_dev = dev_copy.drop(['token', 'label', 'bio_only', 'upos', 'docs'], axis=1)
y_dev = dev_copy['bio_only']

###Train the Random Forest Classifier using the training feature vector and labels

In [ ]:
rfc = RandomForestClassifier().fit(X_train, y_train)

Function to translate the integers 0, 1, 2 to B, I, O

In [ ]:
def index_to_bio(ind):
  bio='M'
  if ind==0:
    bio = 'B'
  elif ind==1:
    bio = 'I'
  elif ind==2:
    bio = 'O'
  return bio

###Get the predictions for the development set from the trained model


In [ ]:
preds = rfc.predict(X_dev)
preds_bio = [index_to_bio(i) for i in preds]
dev_copy['prediction'] = preds_bio

###Compare the number of different values in the golden stanadard label set and the predicted label set. 
B corresponds to 0, I to 1 and O to 2


In [ ]:
y_dev.value_counts()

2    14144
0      826
1      412
Name: bio_only, dtype: int64

In [ ]:
dev_copy['prediction'].value_counts()

O    13289
B     1395
I      698
Name: prediction, dtype: int64

###Analyse the classification report and the confusion matrix

In [ ]:
print(classification_report(y_dev, preds ))

              precision    recall  f1-score   support

           0       0.41      0.66      0.50       826
           1       0.10      0.23      0.14       412
           2       0.98      0.90      0.94     14144

    accuracy                           0.87     15382
   macro avg       0.49      0.60      0.53     15382
weighted avg       0.92      0.87      0.89     15382



i-th row and j-th column entry indicates the number of samples with true label being i-th class and prediced label being j-th class.

```
   B   I   O
   0   1   2
0  tp  ?   fn    
1  ?   tp  fn
2  fp  fp  tn
```


In [ ]:
confusion_matrix(y_dev, preds, labels=[0, 1, 2])

array([[  537,   134,   155],
       [  135,    84,   193],
       [  722,   517, 12905]])

### Import the testing data


In [ ]:
wnuttest = 'https://storage.googleapis.com/wnut-2017_ner-shared-task/wnut17test_clean_tagged.txt'
testset = pd.read_table(wnuttest, header=None, names=['token', 'upos']).dropna()

### Extract the features of the testing data

In [ ]:
test_copy = extract_features(testset, bio=False)

###Prepare the test feature vector and use it with the defined RFC model to get predictions

In [ ]:
X_test = test_copy.drop(['token', 'upos', 'docs'], axis=1)
test_preds = rfc.predict(X_test)
test_preds_bio = [index_to_bio(i) for i in test_preds]
test_copy['prediction'] = test_preds_bio

In [ ]:
test_copy.head(2)

,token,upos,pos_indices,is_propn,title_case,token_length,docs,spacy_tag,is_alpha,is_stop,prediction
0,&,CCONJ,16,False,False,1,&,17571114184892886314,False,False,O
1,gt,X,6,False,False,2,gt,15794550382381185553,True,False,O


### Drop the feature columns and export the test data with the predictions

In [ ]:
test_result = test_copy.drop(['pos_indices',	'is_propn', 'title_case',	'token_length',	'docs',	'spacy_tag',	'is_alpha',	'is_stop'], axis=1)
test_result.head(2)

,token,upos,prediction
0,&,CCONJ,O
1,gt,X,O


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
test_result.to_csv('drive/My Drive/Colab Notebooks/testset_mmm67.csv', index=False)